<a href="https://colab.research.google.com/github/cerberusv2px/nlp-learn/blob/main/11_lstm_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from random import randint
import re



In [2]:
import nltk

nltk.download('gutenberg')
from nltk.corpus import gutenberg as gut

print(gut.fileids())

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
macbeth_text = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')



In [4]:
print(macbeth_text[:500])

[The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lightning. Enter three Witches.

  1. When shall we three meet againe?
In Thunder, Lightning, or in Raine?
  2. When the Hurley-burley's done,
When the Battaile's lost, and wonne

   3. That will be ere the set of Sunne

   1. Where the place?
  2. Vpon the Heath

   3. There to meet with Macbeth

   1. I come, Gray-Malkin

   All. Padock calls anon: faire is foule, and foule is faire,
Houer through 


In [5]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

In [6]:
macbeth_text = preprocess_text(macbeth_text)
macbeth_text[:500]

' the tragedie of macbeth by william shakespeare actus primus scoena prima thunder and lightning enter three witches when shall we three meet againe in thunder lightning or in raine when the hurley burley done when the battaile lost and wonne that will be ere the set of sunne where the place vpon the heath there to meet with macbeth come gray malkin all padock calls anon faire is foule and foule is faire houer through the fogge and filthie ayre exeunt scena secunda alarum within enter king malcom'

### Convert Words into Numbers



In [9]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
macbeth_text_words = word_tokenize(macbeth_text)
n_words = len(macbeth_text_words)
unique_words = len(set(macbeth_text_words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
print('Total Words: %d' % n_words)
print('Unique Words: %d' % unique_words)

Total Words: 17250
Unique Words: 3436


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=unique_words + 1)
tokenizer.fit_on_texts(macbeth_text_words)

In [12]:
vocab_size = len(tokenizer.word_index) + 1
word_2_index = tokenizer.word_index

In [13]:
print(macbeth_text_words[200])
print(word_2_index[macbeth_text_words[200]])

like
69


### Modifying the Shape of Data
Text generation falls in the category of many-to-one sequence problems since the input is a sequence of words and output is a single word. We will be using the Long Short-Term Memory Network (LSTM), which is a type of recurrent neural network to create our text generation model. LSTM accepts data in a 3-dimensional format (number of samples, number of time-steps, features per time-step). Since the output will be a single word, the shape of the output will be 2-dimensional (number of samples, number of unique words in the corpus).



In [14]:
input_sequences = []
output_words = []
input_seq_length = 100

for i in range(0, n_words - input_seq_length, 1):
    in_seq = macbeth_text_words[i:i + input_seq_length]
    out_seq = macbeth_text_words[i + input_seq_length]
    input_sequences.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In the script above, we declare two empty lists input_sequence and output_words. The input_seq_length is set to 100, which means that our input sequence will consist of 100 words. Next, we execute a loop where in the first iteration, integer values for the first 100 words from the text are appended to the input_sequence list. The 101st word is appended to the output_words list. During the second iteration, a sequence of words that starts from the 2nd word in the text and ends at the 101st word is stored in the input_sequence list, and the 102nd word is stored in the output_words array, and so on. A total of 17150 input sequences will be generated since there are 17250 total words in the dataset (100 less than the total words).

In [15]:
print(input_sequences[0])

[1, 869, 4, 40, 60, 1358, 1359, 408, 1360, 1361, 409, 265, 2, 870, 31, 190, 291, 76, 36, 30, 190, 327, 128, 8, 265, 870, 83, 8, 1362, 76, 1, 1363, 1364, 86, 76, 1, 1365, 354, 2, 871, 5, 34, 14, 168, 1, 292, 4, 649, 77, 1, 220, 41, 1, 872, 53, 3, 327, 12, 40, 52, 1366, 1367, 25, 1368, 873, 328, 355, 9, 410, 2, 410, 9, 355, 1369, 356, 1, 1370, 2, 874, 169, 103, 127, 411, 357, 149, 31, 51, 1371, 329, 107, 12, 358, 412, 875, 1372, 51, 20, 170, 92, 9]


normalize our input sequences by dividing the integers in the sequences by the largest integer value. The following script also converts the output into 2-dimensional format.


In [16]:
X = np.reshape(input_sequences, (len(input_sequences), input_seq_length, 1))
X = X / float(vocab_size)

y = to_categorical(output_words)

In [17]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (17150, 100, 1)
y shape: (17150, 3437)


In [18]:
model = Sequential()
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(800, return_sequences=True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation='softmax'))


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 800)          2566400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 800)          5123200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 800)               5123200   
_________________________________________________________________
dense (Dense)                (None, 3437)              2753037   
Total params: 15,565,837
Trainable params: 15,565,837
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
model.fit(X, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
268/268 [==============================] - 44s 163ms/step - loss: 6.7646
Epoch 2/10
268/268 [==============================] - 45s 168ms/step - loss: 6.6684
Epoch 3/10
268/268 [==============================] - 47s 174ms/step - loss: 6.6304
Epoch 4/10
268/268 [==============================] - 48s 180ms/step - loss: 6.6184
Epoch 5/10
268/268 [==============================] - 49s 182ms/step - loss: 6.6093
Epoch 6/10
268/268 [==============================] - 49s 183ms/step - loss: 6.6054
Epoch 7/10
268/268 [==============================] - 49s 184ms/step - loss: 6.6034
Epoch 8/10
268/268 [==============================] - 50s 187ms/step - loss: 6.6068
Epoch 9/10
268/268 [==============================] - 50s 187ms/step - loss: 6.6003
Epoch 10/10
268/268 [==============================] - 50s 186ms/step - loss: 6.5985


To make predictions, we will randomly select a sequence from the `input_sequence` list, convert it into a 3-dimentional shape and then pass it to the `predict()` method of the trained model. The model will return a one-hot encoded array where the index that contains 1 will be the index value of the next word. The index value is then passed to the `index_2_word` dictionary, where the word index is used as a key. The `index_2_word` dictionary will return the word that belong to the index that is passed as a key to the dictionary

In [25]:
random_seq_index = np.random.randint(0, len(input_sequences)-1)
random_seq = input_sequences[random_seq_index]

index_2_word = dict(map(reversed, word_2_index.items()))

word_sequence = [index_2_word[value] for value in random_seq]

print(' '.join(word_sequence))

from my sight and bid my will auouch it yet must not for certaine friends that are both his and mine whose loues may not drop but wayle his fall who my selfe struck downe and thence it is that to your assistance doe make loue masking the businesse from the common eye for sundry weightie reasons murth we shall my lord performe what you command vs murth though our liues macb your spirits shine through you within this houre at most will aduise you where to plant your selues acquaint you with the perfect spy th time the moment


In [26]:
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))
    int_sample = int_sample / float(vocab_size)

    predicted_word_index = model.predict(int_sample, verbose=0)

    predicted_word_id = np.argmax(predicted_word_index)
    seq_in = [index_2_word[index] for index in random_seq]

    word_sequence.append(index_2_word[ predicted_word_id])

    random_seq.append(predicted_word_id)
    random_seq = random_seq[1:len(random_seq)]

In [27]:
final_output = ""
for word in word_sequence:
    final_output = final_output + " " + word

print(final_output)

 from my sight and bid my will auouch it yet must not for certaine friends that are both his and mine whose loues may not drop but wayle his fall who my selfe struck downe and thence it is that to your assistance doe make loue masking the businesse from the common eye for sundry weightie reasons murth we shall my lord performe what you command vs murth though our liues macb your spirits shine through you within this houre at most will aduise you where to plant your selues acquaint you with the perfect spy th time the moment the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
